#### IMPORTS

In [17]:
import os
os.environ['R_HOME'] = 'C:/Users/maceo.valente/AppData/Local/Programs/R/R-4.3.2'

In [18]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import DataFrame, IntVector, StrVector

In [19]:
print("hello")

hello


In [25]:
# Charger le script R depuis un fichier
os.chdir("C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV")
with open("C:/Users/maceo.valente/Documents/Automatisation/TEST_MVA/TSAgriPV/R_related/testVigne.R", 'r') as f:
    r_script = f.read()
robjects.r(r_script)

v Using stics "modulostics" (exe: "stics_modulo.exe")
[1] "Lang_Romain_1"
[1] "[05/06/24]-[16:37:02] INFO - Files generated :"                                                                         
[2] "[05/06/24]-[16:37:02] INFO - \tC:\\Users\\maceo.valente\\Documents\\Automatisation\\exempleVigne\\mod_bLang_Romain_1.sti"
[3] "[05/06/24]-[16:37:02] INFO - \tC:\\Users\\maceo.valente\\Documents\\Automatisation\\exempleVigne\\modhistory.sti"        
v All usms ran successfully!
[1] 9.18
[1] 11.87215
v Using stics "modulostics" (exe: "stics_modulo.exe")
[1] "Lang_Romain_1"
[1] "[05/06/24]-[16:37:06] INFO - Files generated :"                                                                         
[2] "[05/06/24]-[16:37:06] INFO - \tC:\\Users\\maceo.valente\\Documents\\Automatisation\\exempleVigne\\mod_bLang_Romain_1.sti"
[3] "[05/06/24]-[16:37:06] INFO - \tC:\\Users\\maceo.valente\\Documents\\Automatisation\\exempleVigne\\modhistory.sti"        
v All usms ran successfully!
v Using stic

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In dir.create("weatherFiles/") :
R[write to console]:  'weatherFiles' already exists

